In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]

In [3]:
def remove_underscore_after(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def remove_underscore_before(val): # Remove underscore, and keep the part before the underscore
    return val.split('_')[0]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]

df = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df['value'] = pd.to_numeric(df['value'])
df['model'] = df['model'].map(lambda x: '-'.join(x.split('-')[2:]))
models = df['model'].str.split('-').tolist()

# Remove all underscores from our 2d list, keep one list of the param names and one with param vals
model_names_list = map2d(remove_underscore_before, models)
model_names = np.array(model_names_list)
model_val_list = map2d(remove_underscore_after, models)
model_vals = np.array(model_val_list)

In [4]:
model_vals.shape, model_names.shape

((661, 12), (661, 12))

In [5]:
# Splitting model name into more columns
for i in range(model_names.shape[-1]):
    name = model_names[0][i]
    val = model_vals[:,i]
    df[name] = val

cols = sorted(df.columns.tolist())
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
df['ratio'] = pd.to_numeric(df['ratio'])
display(df)

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df_grouped = df.groupby(list(set(cols)-set(['value']))).agg({'value':['mean', 'std', 'count']})

,model,dataset,metric,value,data,ALL,ratio,method,kw,AL,PL,vit,epochs,lr,bs
0,RN50,RSICD-CLS,zeroshot-val-top1,3.93,RS,ALL,0.1,base,none,False,None,False,5,0.0005,64
1,RN50,UCM-CLS,zeroshot-val-top1,5.71,RS,ALL,0.1,base,none,False,None,False,5,0.0005,64
2,RN50,RSICD,image_to_text_R@1,0.09,RS,ALL,0.1,base,none,False,None,False,5,0.0005,64
3,RN50,RSICD,image_to_text_R@5,0.55,RS,ALL,0.1,base,none,False,None,False,5,0.0005,64
4,RN50,RSICD,image_to_text_R@10,1.01,RS,ALL,0.1,base,none,False,None,False,5,0.0005,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,RN50,Sydney,image_to_text_R@5,3.62,RS,ALL,0.1,base,none,False,None,False,15,0.0005,64
657,RN50,Sydney,image_to_text_R@10,7.04,RS,ALL,0.1,base,none,False,None,False,15,0.0005,64
658,RN50,Sydney,text_to_image_R@1,1.21,RS,ALL,0.1,base,none,False,None,False,15,0.0005,64
659,RN50,Sydney,text_to_image_R@5,2.21,RS,ALL,0.1,base,none,False,None,False,15,0.0005,64


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_grouped)

value  \
                                                                                                         mean   
bs  lr       data AL    kw   dataset   metric             model ratio epochs ALL PL   vit   method              
128 0.000005 RS   False none RSICD     image_to_text_R@1  RN50  0.1   5      ALL None False base     1.664000   
                                       image_to_text_R@10 RN50  0.1   5      ALL None False base     9.832000   
                                       image_to_text_R@5  RN50  0.1   5      ALL None False base     5.282000   
                                       text_to_image_R@1  RN50  0.1   5      ALL None False base     1.024000   
                                       text_to_image_R@10 RN50  0.1   5      ALL None False base     8.262000   
                                       text_to_image_R@5  RN50  0.1   5      ALL None False base     4.624000   
                             RSICD-CLS zeroshot-val-top1  RN50  0.1   5      ALL None False base    24.224000   
                             Sydney    image_to_text_R@1  RN50  0.1   5      ALL None False base     1.730000   
                                       image_to_text_R@10 RN50  0.1   5      ALL None False base    12.516000   
                                       image_to_text_R@5  RN50  0.1   5      ALL None False base     6.922000   
                                       text_to_image_R@1  RN50  0.1   5      ALL None False base     1.290000   
                                       text_to_image_R@10 RN50  0.1   5      ALL None False base    12.194000   
                                       text_to_image_R@5  RN50  0.1   5      ALL None False base     7.768000   
                             UCM       image_to_text_R@1  RN50  0.1   5      ALL None False base     0.772000   
                                       image_to_text_R@10 RN50  0.1   5      ALL None False base     6.730000   
                                       image_to_text_R@5  RN50  0.1   5      ALL None False base     3.320000   
                                       text_to_image_R@1  RN50  0.1   5      ALL None False base     0.854000   
                                       text_to_image_R@10 RN50  0.1   5      ALL None False base     6.928000   
                                       text_to_image_R@5  RN50  0.1   5      ALL None False base     4.298000   
                             UCM-CLS   zeroshot-val-top1  RN50  0.1   5      ALL None False base    33.430000   
    0.00005  RS   False none RSICD     image_to_text_R@1  RN50  0.1   5      ALL None False base     3.216000   
                                       image_to_text_R@10 RN50  0.1   5      ALL None False base    25.282000   
                                       image_to_text_R@5  RN50  0.1   5      ALL None False base    13.546000   
                                       text_to_image_R@1  RN50  0.1   5      ALL None False base     3.146000   
                                       text_to_image_R@10 RN50  0.1   5      ALL None False base    22.596000   
                                       text_to_image_R@5  RN50  0.1   5      ALL None False base    12.946000   
                             RSICD-CLS zeroshot-val-top1  RN50  0.1   5      ALL None False base    52.212000   
                             Sydney    image_to_text_R@1  RN50  0.1   5      ALL None False base     2.576000   
                                       image_to_text_R@10 RN50  0.1   5      ALL None False base    17.504000   
                                       image_to_text_R@5  RN50  0.1   5      ALL None False base    10.704000   
                                       text_to_image_R@1  RN50  0.1   5      ALL None False base     2.294000   
                                       text_to_image_R@10 RN50  0.1   5      ALL None False base    17.626000   
                                       text_to_image_R@5  RN50  0.1   5      ALL None False base     9.940000   
                             UCM    

In [7]:
df_grouped.to_csv('eval.csv')

In [8]:
df_2 = df_grouped.reset_index()
# df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
df_2 = df_2.sort_values(['metric', 'dataset', ('value', 'mean')])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_2)

bs        lr data     AL    kw    dataset              metric model  \
                                                                            
101   64    0.0005   RS  False  none      RSICD   image_to_text_R@1  RN50   
40   128    0.0005   RS  False  none      RSICD   image_to_text_R@1  RN50   
100   64    0.0005   RS  False  none      RSICD   image_to_text_R@1  RN50   
0    128  0.000005   RS  False  none      RSICD   image_to_text_R@1  RN50   
60    64  0.000005   RS  False  none      RSICD   image_to_text_R@1  RN50   
20   128   0.00005   RS  False  none      RSICD   image_to_text_R@1  RN50   
80    64   0.00005   RS  False  none      RSICD   image_to_text_R@1  RN50   
47   128    0.0005   RS  False  none     Sydney   image_to_text_R@1  RN50   
115   64    0.0005   RS  False  none     Sydney   image_to_text_R@1  RN50   
114   64    0.0005   RS  False  none     Sydney   image_to_text_R@1  RN50   
7    128  0.000005   RS  False  none     Sydney   image_to_text_R@1  RN50   
67    64  0.000005   RS  False  none     Sydney   image_to_text_R@1  RN50   
27   128   0.00005   RS  False  none     Sydney   image_to_text_R@1  RN50   
87    64   0.00005   RS  False  none     Sydney   image_to_text_R@1  RN50   
126   64    0.0005   RS  False  none        UCM   image_to_text_R@1  RN50   
53   128    0.0005   RS  False  none        UCM   image_to_text_R@1  RN50   
127   64    0.0005   RS  False  none        UCM   image_to_text_R@1  RN50   
13   128  0.000005   RS  False  none        UCM   image_to_text_R@1  RN50   
73    64  0.000005   RS  False  none        UCM   image_to_text_R@1  RN50   
93    64   0.00005   RS  False  none        UCM   image_to_text_R@1  RN50   
33   128   0.00005   RS  False  none        UCM   image_to_text_R@1  RN50   
103   64    0.0005   RS  False  none      RSICD  image_to_text_R@10  RN50   
41   128    0.0005   RS  False  none      RSICD  image_to_text_R@10  RN50   
102   64    0.0005   RS  False  none      RSICD  image_to_text_R@10  RN50   
1    128  0.000005   RS  False  none      RSICD  image_to_text_R@10  RN50   
61    64  0.000005   RS  False  none      RSICD  image_to_text_R@10  RN50   
21   128   0.00005   RS  False  none      RSICD  image_to_text_R@10  RN50   
81    64   0.00005   RS  False  none      RSICD  image_to_text_R@10  RN50   
48   128    0.0005   RS  False  none     Sydney  image_to_text_R@10  RN50   
117   64    0.0005   RS  False  none     Sydney  image_to_text_R@10  RN50   
116   64    0.0005   RS  False  none     Sydney  image_to_text_R@10  RN50   
8    128  0.000005   RS  False  none     Sydney  image_to_text_R@10  RN50   
68    64  0.000005   RS  False  none     Sydney  image_to_text_R@10  RN50   
28   128   0.00005   RS  False  none     Sydney  image_to_text_R@10  RN50   
88    64   0.00005   RS  False  none     Sydney  image_to_text_R@10  RN50   
54   128    0.0005   RS  False  none        UCM  image_to_text_R@10  RN50   
129   64    0.0005   RS  False  none        UCM  image_to_text_R@10  RN50   
128   64    0.0005   RS  False  none        UCM  image_to_text_R@10  RN50   
14   128  0.000005   RS  False  none        UCM  image_to_text_R@10  RN50   
74    64  0.000005   RS  False  none        UCM  image_to_text_R@10  RN50   
34   128   0.00005   RS  False  none        UCM  image_to_text_R@10  RN50   
94    64   0.00005   RS  False  none        UCM  image_to_text_R@10  RN50   
42   128    0.0005   RS  False  none      RSICD   image_to_text_R@5  RN50   
105   64    0.0005   RS  False  none      RSICD   image_to_text_R@5  RN50   
104   64    0.0005   RS  False  none      RSICD   image_to_text_R@5  RN50   
2    128  0.000005   RS  False  none      RSICD   image_to_text_R@5  RN50   
62    64  0.000005   RS  False  none      RSICD   image_to_text_R@5  RN50   
22   128   0.00005   RS  False  none      RSICD   image_to_text_R@5  RN50   
82    64   0.00005   RS  False  none      RSICD   image_to_text_R@5  RN50   
49   128    0.0005   RS  False  none     Sydney   image_to_text_R@5  RN50   
11